In [3]:
import pandas as pd
import numpy as np
import re
from umap import UMAP
from math import log
import os
from tqdm import tqdm
from stop_words import get_stop_words
stop_words = get_stop_words('ru')
from pymystem3 import Mystem
from collections import defaultdict
import artm
import math
from scipy.spatial import distance
%matplotlib inline
import glob
import gensim
from sklearn.mixture import GaussianMixture
import logging
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
import hdbscan
from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer
import matplotlib.pyplot as plt
from statistics import median, variance
import pickle
import warnings
warnings.filterwarnings('ignore')
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)

# Functions

In [4]:
pattern = re.compile("[а-яА-ЯёЁ]+")
m = Mystem()
model_path = r'C:\Users\vdtri\Documents\ВКР\Программа\models\\'
path_to_prog = r'C:\Users\vdtri\Documents\ВКР\Программа\\'

def stemming(text, m):
    lemmas = m.lemmatize(text)
    return ''.join(lemmas)

def sem_close_ARTM(model_artm):
    sem = list()
    for topic_name in model_artm.topic_names:
#         print(topic_name + ': ')
        temp = model_artm.score_tracker['TopTokensScore'].last_tokens[topic_name]
        sum_ = 0
        for ind, word1 in enumerate(temp):
            for word2 in temp:
                try:
                    sum_ += distance.cosine(w2v[word1], w2v[word2])
                except:
                    sum_ +=1.0
        sem.append(sum_/len(temp))
#    return sum(sem)/len(sem)
    return sem

def doc2vow(text):
    text = text.split(sep='.')
    temp_block = list()
    for pred in text:
        temp_block.append(' '.join(re.findall(pattern, pred)))
    text = temp_block   
    text = stemming(' . '.join(text), m).split(sep = '.')
    
    vow = str()
    defdict = defaultdict(int)
    new_text = str()
    for word in''.join(text).replace('\n', '').split():
        if len(word)>2 and word not in stop_words:
            defdict[word] += 1
    vow += 'doc' + ' '
    for key, elem in defdict.items():
        vow += key + ':' + str(elem) + ' '
    vow += '\n'
    with open('vow_test.txt', 'w+', encoding='utf-8') as file:
        file.write(vow)
    batch_vectorizer_test = artm.BatchVectorizer(data_path='vow_test.txt',
                                        data_format='vowpal_wabbit',
                                        target_folder='batch_test')
    return batch_vectorizer_test

def get_theta(model, batch):
    return model_artm.transform(batch_vectorizer=batch)

def bestTopics(vector, topics):
    new_vec = list()
    for ind, item in enumerate(topics):
        new_vec.append(vector[topics[ind]])
    return new_vec

def sem_close_Top2vec(model):
    sem = list()
    for i in range(model.get_num_topics()):
        temp = model.topic_words[i][:6]
        sum_ = 0
        for ind, word1 in enumerate(temp):
            for word2 in temp:
                try:
                    sum_ += distance.cosine(navec[word1], navec[word2])
                except:
                    sum_ +=0.7
        sem.append(sum_/len(temp))
#    return sum(sem)/len(sem)
    return sem

def topic_probs(doc, topic_vecs, k):
    results = list()
    for i in topic_vecs:
        results.append(1/(distance.euclidean(doc, i) ** k))
    return np.array(results)/sum(results)

# def dist_top10(item, items_list):
#     res = list()
#     for i in tqdm(items_list):
#         res.append(distance.cosine(item, list(i)))
#     return sorted(zip(range(len(items_list)), res), key = lambda x: x[1])

def dist_top10(item, items_list):
    res = list()
    for i in items_list:
        res.append(distance.cosine(item, i))
    return sorted(zip(range(len(res)), res), key = lambda x: x[1])[1:11]

def prec(pred, k, num):
    s = 0
    for i in range(k):
        if pred[i] < num + 11 and pred[i]>num:
            s += 1
    return s/k

def clean_data(text):
    text = text.split(sep='.')
    temp_block = list()
    for pred in text:
        temp_block.append(' '.join(re.findall(pattern, pred)))
    text = temp_block   
    text = stemming(' . '.join(text), m).split(sep = '.')
    new_text = str()
    for word in ''.join(text).replace('\n', '').split():
        if len(word)>2 and word not in stop_words:
            new_text += word + ' '
    return new_text

def create_dict(texts):
    list_of_dicts = list()
    dict_ = defaultdict(int)
    for text in tqdm(texts):
        list_of_dicts.append(defaultdict(int))
        for word in text.split():
            list_of_dicts[-1][word] += 1
            dict_[word] += 1
    return (list_of_dicts, dict_)

# Preprocessing

In [4]:
os.chdir(r'C:\Users\vdtri\Documents\ВКР\Программа\wiki_18k')

texts = list()
path = r'C:\Users\vdtri\Documents\ВКР\Программа\wiki_18k'
for file in tqdm(os.listdir(path)):
    with open(file, 'r', encoding='utf-8') as file_handler:
        texts.append(file_handler.read())
texts = list(set(texts))

In [5]:
os.chdir(r'C:\Users\vdtri\Documents\ВКР\Программа')
habr = pd.read_csv("habr_posts.csv")
habr = list(habr['text'].values)
habr_texts = list()
for i in tqdm(habr):
    try:
        if len(i.split())>500:
            habr_texts.append(i)
    except:
        pass
habr_texts = list(set(habr_texts))
len(habr_texts)

In [6]:
all_texts = list()
all_texts = texts + habr_texts
print(len(all_texts))

In [7]:
for i, item in enumerate(all_texts):
    all_texts[i] = all_texts[i].split(sep='.')
for i in tqdm(range(len(all_texts))):
    temp_block = list()
    for pred in all_texts[i]:
        temp_block.append(' '.join(re.findall(pattern, pred)))
    new_str = str()
    for word in ' '.join(temp_block).split():
        if word not in stop_words and len(word)>2:
            new_str = new_str + word + ' '
    all_texts[i] = new_str

In [8]:
for i in tqdm(range(len(all_texts))):
    temp_block = list() 
    all_texts[i] = stemming(' . '.join(all_texts[i]), m).split(sep = '.')

with open('entry.pickle', 'wb') as f: pickle.dump(all_texts, f)

Vowpal wabbit

In [9]:
vow_list = list()
for i in tqdm(range(len(all_texts_clean))):
    vow = str()
    defdict = defaultdict(int)
    new_text = str()
    for word in all_texts_clean[i].replace('\n', '').split():
        if len(word)>2 and word not in stop_words:
            defdict[word] += 1
    vow += 'doc' + str(i) + ' '
    for key, elem in defdict.items():
        vow += key + ':' + str(elem) + ' '
    vow_list.append(vow)
#    vow += '\n'

with open('vow.txt', 'w+', encoding='utf-8') as file_handler:
    for i in tqdm(range(len(vow_list))):
        file_handler.write(vow_list[i] + '\n')

Uci bag of words

In [12]:
list_of_dicts, dict_ = create_dict(all_texts_clean)

wordID = defaultdict(int)
i = 1
for key, value in list(dict_.items()):
    wordID[key] = i
    i += 1
    
triplet_list = list()
for i, text in enumerate(all_texts_clean):
    for word in text.split():
        triplet_list.append((i, wordID[word], list_of_dicts[i][word]))
len(triplet_list)

with open('docword.UCI_dataset', mode = 'w+', encoding='utf-8') as file_handler:
    file_handler.write(str(len(all_texts_clean)) + '\n')
    file_handler.write(str(len(dict_.keys())) + '\n')  
    sum = 0
    for key, value in list(dict_.items()):
        sum += value
    file_handler.write(str(sum) + '\n') 
    for item1, item2, item3 in tqdm(triplet_list):
        file_handler.write(str(item1) + ' ' + str(item2) + ' ' + str(item3) + '\n')

with open(r'C:\Users\vdtri\Documents\ВКР\Программа\vocab.UCI_dataset.txt', mode = 'w+', encoding='utf-8') as file_handler: 
    for key, value in list(dict_.items()): 
        file_handler.write(str(key) + '\n')

In [14]:
UCI_data = str()
with open('UCI_dataset', mode = 'r', encoding='utf-8') as file_handler:
    UCI_data = file_handler.read()

# SCDV

In [5]:
class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

epoch_logger = EpochLogger()

In [6]:
class SCDV:
    
    def __init__(self, data, w2v):
        self.data = data
        self.w2v = w2v
        self.gmm_word = GaussianMixture(n_components=120,
                                        max_iter = 50)
        self.gmm_text = GaussianMixture(n_components=180, 
                                        max_iter = 80, 
                                        covariance_type = 'tied')
        umap_default_args = {'n_neighbors': 30,
             'n_components': 15,
             'metric': 'cosine'}
        self.n_features = 120
        self.umap_word = UMAP(**umap_default_args)
        self.umap_text = UMAP(**umap_default_args)
        
    def fit(self,
            tune_umap_word = False, 
            tune_gmm_word = False, 
            tune_umap_text = False,
            tune_gmm_text = False,  
            val_data = None):
        self.idfs = self._get_idfs()
        self.keys= list()
        self.word_vectors = list()
        for i, _ in tqdm(self.idfs.items()):
            try:
                self.word_vectors.append(w2v[i])
                self.keys.append(i)
            except:
                pass
        if tune_umap_word:
            self._tune_umap_word(val_data)
        else:
            self.umap_word.fit(np.array(self.word_vectors))
        if tune_gmm_word:
            self._tune_gmm_word(val_data)
        else:
            self.gmm_word.fit(self.umap_word.embedding_)
        self.soft_clusters = self._get_soft_clusters()
        self.soft_clustersKD = self._get_soft_clustersKD()
        if tune_umap_text:
            self._tune_umap_text(val_data)
        if tune_gmm_text:
            self._tune_gmm_text(val_data)
        
        text_vectors = list()
        length = len(self.umap_word.embedding_[0])*self.n_features
        for elem in self.data:
            sum_ = np.array([0]*length)
            counter = 0
            for word in elem.split():
                try:
                    sum_ = sum_ + np.array(self.soft_clustersKD[word])
                    counter = counter + 1
                except:
                    pass
            text_vectors.append(sum_/counter)

        counter = 0
        i = 1
        for counter, elem in enumerate(text_vectors):
            i = 1
            for a in elem:
                if math.isnan(a):
                    text_vectors[counter] = [0]*length
        if tune_umap_text == False:
            self.umap_text.fit(text_vectors)
        if tune_gmm_text == False:
            self.gmm_text.fit(self.umap_text.embedding_)
    
    def _get_soft_clusters(self):
        pred = self.gmm_word.predict_proba(self.umap_word.embedding_)
        soft_clusters = dict(zip(self.keys, pred))
        return soft_clusters
        
    def _get_soft_clustersKD(self):
        i = 0
        soft_clustersKD = defaultdict(list)
        for key in self.keys:
            new_list = list()
            for k in self.soft_clusters[key]:
                new_list = new_list + list(np.array(self.umap_word.embedding_[i])*k)
            soft_clustersKD[key] = np.array(new_list)*self.idfs[key]
            i = i + 1
        return soft_clustersKD
        
        
    def _get_idfs(self):
        tfv = TfidfVectorizer(min_df = 2)
        tfidfmatrix_traindata = tfv.fit_transform(self.data)
        featurenames = tfv.get_feature_names()
        idf = tfv._tfidf.idf_
        return dict(zip(featurenames, idf))        
        
    def _tune_umap_text(self, val_data):
        print("tune umap text\n")
        scores = list()        
        text_vectors = list()
        length = len(self.umap_word.embedding_[0])*self.n_features
        for elem in self.data:
            sum_ = np.array([0]*length)
            counter = 0
            for word in elem.split():
                try:
                    sum_ = sum_ + np.array(self.soft_clustersKD[word])
                    counter = counter + 1
                except:
                    pass
            text_vectors.append(sum_/counter)

        counter = 0
        i = 1
        for counter, elem in enumerate(text_vectors):
            i = 1
            for a in elem:
                if math.isnan(a):
                    text_vectors[counter] = [0]*length
                    
        for i in tqdm(range(2, 15)):
            umap_args = {'n_neighbors': 30,
                 'n_components': i,
                 'metric': 'cosine'}
            self.umap_text = UMAP(**umap_args)
            self.umap_text.fit(text_vectors)
            self.gmm_text.fit(self.umap_text.embedding_)
            va = self._compute_score_pred(val_data)
            print(va)
            scores.append(va)
        param = sorted(zip(scores, range(2, 15)), 
                       key = lambda x: x[0], 
                       reverse = True)[0][1]
        umap_args = {'n_neighbors': 30,
                     'n_components': param,
                     'metric': 'cosine'}
        self.umap_text.fit(np.array(text_vectors))
        
    def _tune_umap_word(self, val_data):
        print("tune umap word\n")
        scores = list()
        for i in tqdm(range(2, 15)):
            umap_args = {'n_neighbors': 30,
                 'n_components': i,
                 'metric': 'cosine'}
            self.umap_word = UMAP(**umap_args)
            self.umap_word.fit(np.array(self.word_vectors))
            self.gmm_word.fit(self.umap_word.embedding_)
            self.soft_clusters = self._get_soft_clusters()
            self.soft_clustersKD = self._get_soft_clustersKD() 
            
            text_vectors = list()
            length = len(self.umap_word.embedding_[0])*self.n_features
            for elem in self.data:
                sum_ = np.array([0]*length)
                counter = 0
                for word in elem.split():
                    try:
                        sum_ = sum_ + np.array(self.soft_clustersKD[word])
                        counter = counter + 1
                    except:
                        pass
                text_vectors.append(sum_/counter)
                
            counter = 0
            i = 1
            for counter, elem in enumerate(text_vectors):
                i = 1
                for a in elem:
                    if math.isnan(a):
                        text_vectors[counter] = [0]*length
            self.umap_text.fit(text_vectors)
            self.gmm_text.fit(self.umap_text.embedding_)
            print("brefore score")
            va = self._compute_score_pred(val_data)
            print(va)
            scores.append(va)
        param = sorted(zip(scores, range(2, 15)), 
                       key = lambda x: x[0], 
                       reverse = True)[0][1]
        umap_args = {'n_neighbors': 30,
                     'n_components': param,
                     'metric': 'cosine'}
        self.umap_word.fit(np.array(self.word_vectors))
            
        
    def _tune_gmm_word(self, val_data):
        print("tune gmm word\n")
        scores = list()
        for i in tqdm(range(20, 140, 20)):
            self.gmm_word = GaussianMixture(
                n_components=i,
                max_iter = 50)
            self.n_features = i
            self.gmm_word.fit(self.umap_word.embedding_)
            self.soft_clusters = self._get_soft_clusters()
            self.soft_clustersKD = self._get_soft_clustersKD() 
            
            text_vectors = list()
            length = len(self.umap_word.embedding_[0])*self.n_features
            for elem in self.data:
                sum_ = np.array([0]*length)
                counter = 0
                for word in elem.split():
                    try:
                        sum_ = sum_ + np.array(self.soft_clustersKD[word])
                        counter = counter + 1
                    except:
                        pass
                text_vectors.append(sum_/counter)
                
            counter = 0
            i = 1
            for counter, elem in enumerate(text_vectors):
                i = 1
                for a in elem:
                    if math.isnan(a):
                        text_vectors[counter] = [0]*length
            self.umap_text.fit(text_vectors)
            self.gmm_text.fit(self.umap_text.embedding_)
            va = self._compute_score_pred(val_data)
            print(va)
            scores.append(va)
        param = sorted(zip(scores, range(20, 140, 20)), 
                       key = lambda x: x[0], 
                       reverse = True)[0][1]
        self.gmm_word = GaussianMixture(
            n_components=param,
            max_iter = 50)
        self.gmm_word.fit(self.umap_word.embedding_)
        
    def _tune_gmm_text(self, val_data):
        print("tune gmm text\n")
        scores = list()        
        text_vectors = list()
#        length = len(w2v['диск'])*len(self.soft_clustersKD["диск"])
        length = len(self.umap_word.embedding_[0])*self.n_features
        for elem in self.data:
            sum_ = np.array([0]*length)
            counter = 0
            for word in elem.split():
                try:
                    sum_ = sum_ + np.array(self.soft_clustersKD[word])
                    counter = counter + 1
                except:
                    pass
            text_vectors.append(sum_/counter)

        counter = 0
        i = 1
        for counter, elem in enumerate(text_vectors):
            i = 1
            for a in elem:
                if math.isnan(a):
                    text_vectors[counter] = [0]*length
                    
        for i in tqdm(range(40, 220, 20)):
            self.gmm_text = GaussianMixture(n_components=i, 
                                max_iter = 80, 
                                covariance_type = 'tied')
            self.gmm_text.fit(self.umap_text.embedding_)
            va = self._compute_score(val_data)
            print(va)
            scores.append(va)
        param = sorted(zip(scores, range(40, 220, 20)), 
                       key = lambda x: x[0], 
                       reverse = True)[0][1]
        self.gmm_text = GaussianMixture(n_components=param, 
                            max_iter = 80, 
                            covariance_type = 'tied')
        self.gmm_text.fit(self.umap_text.embedding_)
        
    def _compute_score(self, val_data):
        test_vectors = list()
        length = len(self.umap_word.embedding_[0])*self.n_features
        for text in val_data:
            sum_ = np.array([0]*length)
            counter = 0
            for word in text.split():
                try:
                    sum_ = sum_ + np.array(self.soft_clustersKD[word])
                    counter = counter + 1
                except:
                    pass
            test_vectors.append(sum_/counter)
        for counter, elem in enumerate(test_vectors):
            for a in elem:
                if (math.isnan(a)):
                    test_vectors[counter] = [0]*length

        pred_prob = self.umap_text.transform(test_vectors)
        for counter, elem in enumerate(pred_prob):
            for a in elem:
                if math.isnan(a):
                    pred_prob[counter] = [0]*len(self.umap_text.embedding_[0])
        new_test_vecs = self.gmm_text.predict_proba(pred_prob)
        
        big_res = list()
        for s in range(len(val_data) // 100):
            results = list()
            for i in [i*11 for i in range(10)]:
                results.append(dist_top10(new_test_vecs[110*s:110*(s+1)][i], 
                                          new_test_vecs[110*s:110*(s+1)]))

            leest = list()
            for i in range(10):
                leest.append(prec(list(map(lambda x: x[0], results[i])), 10, i*11))
            big_res.append(sum(leest)/len(leest))
        return sum(big_res)/len(big_res)
    
    def _compute_score_pred(self, val_data):
        test_vectors = list()
        
        length = len(self.umap_word.embedding_[0])*self.n_features
        
        for text in val_data:
            sum_ = np.array([0]*length)
            counter = 0
            for word in text.split():
                try:
                    sum_ = sum_ + np.array(self.soft_clustersKD[word])
                    counter = counter + 1
                except:
                    pass
            test_vectors.append(sum_/counter)
        for counter, elem in enumerate(test_vectors):
            for a in elem:
                if (math.isnan(a)):
                    test_vectors[counter] = [0]*length
            

        pred_prob = self.umap_text.transform(test_vectors)
        
        big_res = list()
        for s in range(len(val_data) // 100):
            results = list()
            for i in [i*11 for i in range(10)]:
                results.append(dist_top10(pred_prob[110*s:110*(s+1)][i], 
                                          pred_prob[110*s:110*(s+1)]))

            leest = list()
            for i in range(10):
                leest.append(prec(list(map(lambda x: x[0], results[i])), 10, i*11))
            big_res.append(sum(leest)/len(leest))
        return sum(big_res)/len(big_res)
    
    def get_vectors(self, texts):
        vectors = list()
        length = len(self.umap_word.embedding_[0])*self.n_features
        for text in texts:
            sum_ = np.array([0]*length)
            counter = 0
            for word in text.split():
                try:
                    sum_ = sum_ + np.array(self.soft_clustersKD[word])
                    counter = counter + 1
                except:
                    pass
            vectors.append(sum_/counter)
        for counter, elem in enumerate(vectors):
            for a in elem:
                if (math.isnan(a)):
                    vectors[counter] = [0]*length
        pred_prob = self.umap_text.transform(vectors)
        new_test_vecs_1 = self.gmm_text.predict_proba(pred_prob)
        return new_test_vecs_1


In [7]:
from gensim.models.fasttext import FastTextKeyedVectors
from gensim.models import KeyedVectors
w2v = FastTextKeyedVectors.load(r'C:\Users\vdtri\Documents\ВКР\Программа\213\\' + 'model.model')
#w2v = KeyedVectors.load(model_path + 'word2vec_hard_06.model')

In [8]:
with open(r'C:\Users\vdtri\Documents\ВКР\Программа\models\SCDV\sen_1', 'rb') as f:
    sentences_1 = pickle.load(f)

In [9]:
val = sentences_1[-550:]

In [10]:
scdv = SCDV(sentences_1, w2v)

In [ ]:
scdv.fit(True, True, True, True, val)

In [ ]:
with open(r'C:\Users\vdtri\Documents\ВКР\Программа\models\SCDV\scdv_obj', 'wb') as f: 
    pickle.dump(scdv, f)

In [109]:
test_vectors = list()
for elem in val[:110]:
    test_vectors.append(SCDV.get_vector(elem))

NameError: name 'self' is not defined

# ARTM

In [23]:
os.chdir(r'C:\Users\vdtri\Documents\ВКР\Программа')
data_path = r''
batch_vectorizer = artm.BatchVectorizer(data_path='', data_format='bow_uci',
                                        collection_name='UCI_dataset', target_folder='kos_batches')
dictionary = batch_vectorizer.dictionary
type(dictionary)

In [24]:
dictionary = batch_vectorizer.dictionary
score_ret = list()
scores_1 = list()
for T in tqdm(range(30, 310, 20)):
    topic_names = ['topic_{}'.format(i) for i in range(T)]
    model_artm = artm.ARTM(topic_names=topic_names, cache_theta=True, 
                           scores=[artm.PerplexityScore(name='PerplexityScore', dictionary=dictionary)], 
                           regularizers=[artm.SmoothSparseThetaRegularizer(name='SparseTheta', tau=-0.15)])
    model_artm.scores.add(artm.SparsityPhiScore(name='SparsityPhiScore'))
    model_artm.scores.add(artm.SparsityThetaScore(name='SparsityThetaScore'))
    model_artm.scores.add(artm.TopicKernelScore(name='TopicKernelScore',
                                                      probability_mass_threshold=0.3))
    model_artm.regularizers.add(artm.SmoothSparsePhiRegularizer(name='SparsePhi', tau=-0.1))
    model_artm.regularizers.add(artm.DecorrelatorPhiRegularizer(name='DecorrelatorPhi', tau=1.5e+5))
    model_artm.scores.add(artm.TopTokensScore(name='TopTokensScore', num_tokens=6))
    model_artm.initialize(dictionary=dictionary)
    model_artm.num_document_passes = 1
    model_artm.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=8)
    
    s = list()
    for ind in range(5):
        theta_list_test = list()
        for i in (test_data[110*ind:(ind+1)*110]):
            theta_list_test.append(get_theta(model_artm, doc2vow(i)))
        results = list()
        for i in [i*11 for i in range(5)]:
            results.append(dist_top10(theta_list_test[i], theta_list_test))
        leest = list()
        for i in range(5):
            leest.append(prec(list(map(lambda x: x[0], results[i])), 10, i*11))
        s.append(sum(leest)/len(leest))
    score_ret.append(sum(s)/len(s))
    print(sum(s)/len(s))
         
#     model_artm.fit_online(batch_vectorizer=batch_vectorizer, asynchronous=True)
    scores_1.append((T, model_artm.score_tracker['PerplexityScore'].last_value, sem_close_ARTM(model_artm)))
#    print((T, model_artm.score_tracker['PerplexityScore'].last_value))

In [25]:
dictionary = batch_vectorizer.dictionary
T = 90
topic_names = ['topic_{}'.format(i) for i in range(T)]
model_artm = artm.ARTM(topic_names=topic_names, cache_theta=True, 
                       scores=[artm.PerplexityScore(name='PerplexityScore', dictionary=dictionary)], 
                       regularizers=[artm.SmoothSparseThetaRegularizer(name='SparseTheta', tau=-0.15)])
model_artm.scores.add(artm.SparsityPhiScore(name='SparsityPhiScore'))
model_artm.scores.add(artm.SparsityThetaScore(name='SparsityThetaScore'))
model_artm.scores.add(artm.TopTokensScore(name='TopTokensScore', num_tokens=6))
model_artm.regularizers.add(artm.SmoothSparsePhiRegularizer(name='SparsePhi', tau=-0.1))
model_artm.regularizers.add(artm.DecorrelatorPhiRegularizer(name='DecorrelatorPhi', tau=1.5e+5))
model_artm.initialize(dictionary=dictionary)
model_artm.num_document_passes = 3
model_artm.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=15)
#    model_artm.fit_online(batch_vectorizer=batch_vectorizer, asynchronous=True)
print(model_artm.score_tracker['PerplexityScore'].last_value)

In [26]:
for topic_name in model_artm.topic_names:
    print(topic_name + ': ')
    print(model_artm.score_tracker['TopTokensScore'].last_tokens[topic_name])

In [27]:
i=0

# Top2vec

In [29]:
import top2vec
from top2vec import Top2Vec

In [ ]:
umap_args = {'n_neighbors': 30,
             'n_components': 15,
             'metric': 'cosine'}
model = Top2Vec(documents=all_texts_clean, min_count=0, speed="deep-learn", workers=4)

In [ ]:
model = Top2Vec.load(model_path + 'top2vec_deep.model')

In [ ]:
num = 0
class_0 = list()
class_0_vectors = list()
for i in range(len(model._get_document_vectors())):
    if (model.get_documents_topics([i])[0][0] == num):
        class_0.append(i)

for i, val in enumerate(model._get_document_vectors()):
    if (i in class_0):
        class_0_vectors.append(np.array(val))